<a href="https://colab.research.google.com/github/shashi3876/machineLearning/blob/main/Updated_NHL_API_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get all available season IDs

In [ ]:
import requests

res=requests.get('https://api-web.nhle.com/v1/standings-season')

seasonIds=[]
for seasonId in res.json()['seasons']:
  seasonIds.append(seasonId['id'])
print(seasonIds)
print(len(seasonIds))

[19171918, 19181919, 19191920, 19201921, 19211922, 19221923, 19231924, 19241925, 19251926, 19261927, 19271928, 19281929, 19291930, 19301931, 19311932, 19321933, 19331934, 19341935, 19351936, 19361937, 19371938, 19381939, 19391940, 19401941, 19411942, 19421943, 19431944, 19441945, 19451946, 19461947, 19471948, 19481949, 19491950, 19501951, 19511952, 19521953, 19531954, 19541955, 19551956, 19561957, 19571958, 19581959, 19591960, 19601961, 19611962, 19621963, 19631964, 19641965, 19651966, 19661967, 19671968, 19681969, 19691970, 19701971, 19711972, 19721973, 19731974, 19741975, 19751976, 19761977, 19771978, 19781979, 19791980, 19801981, 19811982, 19821983, 19831984, 19841985, 19851986, 19861987, 19871988, 19881989, 19891990, 19901991, 19911992, 19921993, 19931994, 19941995, 19951996, 19961997, 19971998, 19981999, 19992000, 20002001, 20012002, 20022003, 20032004, 20052006, 20062007, 20072008, 20082009, 20092010, 20102011, 20112012, 20122013, 20132014, 20142015, 20152016, 20162017, 20172018,

# Get all team codes

In [ ]:
res=requests.get('https://api.nhle.com/stats/rest/en/team')
teamId=[]
teamDict={}
for team in res.json()['data']:
  teamId.append((team['rawTricode'],team['franchiseId'],team['fullName']))
  teamDict[team['rawTricode']]=team['fullName']
print(res.json())
print(teamId)
print(teamDict)
print(len(teamId))

{'data': [{'id': 11, 'franchiseId': 35, 'fullName': 'Atlanta Thrashers', 'leagueId': 133, 'rawTricode': 'ATL', 'triCode': 'ATL'}, {'id': 34, 'franchiseId': 26, 'fullName': 'Hartford Whalers', 'leagueId': 133, 'rawTricode': 'HFD', 'triCode': 'HFD'}, {'id': 31, 'franchiseId': 15, 'fullName': 'Minnesota North Stars', 'leagueId': 133, 'rawTricode': 'MNS', 'triCode': 'MNS'}, {'id': 32, 'franchiseId': 27, 'fullName': 'Quebec Nordiques', 'leagueId': 133, 'rawTricode': 'QUE', 'triCode': 'QUE'}, {'id': 33, 'franchiseId': 28, 'fullName': 'Winnipeg Jets (1979)', 'leagueId': 133, 'rawTricode': 'WIN', 'triCode': 'WIN'}, {'id': 35, 'franchiseId': 23, 'fullName': 'Colorado Rockies', 'leagueId': 133, 'rawTricode': 'CLR', 'triCode': 'CLR'}, {'id': 36, 'franchiseId': 3, 'fullName': 'Ottawa Senators (1917)', 'leagueId': 133, 'rawTricode': 'SEN', 'triCode': 'SEN'}, {'id': 37, 'franchiseId': 4, 'fullName': 'Hamilton Tigers', 'leagueId': 133, 'rawTricode': 'HAM', 'triCode': 'HAM'}, {'id': 38, 'franchiseId':

# Get Roaster for each year

In [ ]:
import pandas as pd
player_data = []

for team in teamDict.keys():
    for season in seasonIds:
        url = f'https://api-web.nhle.com/v1/roster/{team}/{season}'
        res = requests.get(url)

        if res.status_code == 200:
            #print(f'Got for team {team} for season {season}')
            roster_data = res.json()

            for position in ['forwards', 'defensemen', 'goalies']:
                for player in roster_data[position]:
                    player_info = {
                        'Team': team,
                        'Position': player['positionCode'],
                        'FirstName': player['firstName']['default'],
                        'LastName': player['lastName']['default'],
                        'ID': player['id']
                    }
                    player_data.append(player_info)

# Create a DataFrame from the list of player data
df = pd.DataFrame(player_data)

# Display the DataFrame
print(df)

      Team Position FirstName   LastName       ID
0      ATL        L     Bryan      Adams  8467867
1      ATL        R    Donald    Audette  8445176
2      ATL        L      Eric   Bertrand  8460014
3      ATL        L     Jason  Botterill  8460510
4      ATL        L    Andrew   Brunette  8459596
...    ...      ...       ...        ...      ...
54882  OTT        D      Jake  Sanderson  8482105
54883  OTT        D     Artem        Zub  8482245
54884  OTT        G     Anton   Forsberg  8476341
54885  OTT        G    Joonas  Korpisalo  8476914
54886  OTT        G      Mads    Sogaard  8481544

[54887 rows x 5 columns]


In [ ]:
# Group by player ID and aggregate
agg_df = df.groupby('ID').agg({
    'FirstName': 'first',
    'LastName': 'first',
    'Team': lambda x: list(set(x)),  # Use a set to remove duplicates and then convert to a list
    'Position': lambda x: list(set(x)),  # Use a set to remove duplicates and then convert to a list
}).reset_index()

# Rename the columns for clarity
agg_df.columns = ['ID', 'FirstName', 'LastName', 'Teams', 'Positions']

# Calculate the number of teams for each player
agg_df['NumTeams'] = agg_df['Teams'].apply(lambda x: len(x))


# Display the aggregated DataFrame
print(agg_df)


           ID FirstName     LastName       Teams Positions  NumTeams
0     8444850     Henry       Harris       [BOS]       [R]         1
1     8444851    Gordon       Spence       [TSP]       [L]         1
2     8444852       Ron       Hudson       [DET]       [C]         1
3     8444853      Norm      Johnson  [CHI, BOS]       [C]         2
4     8444854       Reg       Abbott       [MTL]       [C]         1
...       ...       ...          ...         ...       ...       ...
8464  8484255     Jason        Polin       [COL]       [R]         1
8465  8484256      Jake  Livingstone       [NSH]       [D]         1
8466  8484258       Sam     Malinski       [COL]       [D]         1
8467  8484287      Cole       McWard       [VAN]       [D]         1
8468  8484293     Yaniv       Perets       [CAR]       [G]         1

[8469 rows x 6 columns]


In [ ]:
# Group by first and last name, aggregate ID as a list, and count
agg_df_2 = agg_df.groupby(['FirstName', 'LastName']).agg({'ID': ['count', list]}).reset_index()

# Rename the columns for clarity
agg_df_2.columns = ['FirstName', 'LastName', 'Count', 'IDList']

# Sort the DataFrame by the Count column in decreasing order
agg_df_2 = agg_df_2.sort_values(by='Count', ascending=False)

# Display the resulting DataFrame
print(agg_df_2.head(30))

      FirstName    LastName  Count                       IDList
5826      Mikko    Lehtonen      3  [8469714, 8471747, 8482247]
4219       John       Adams      2           [8444862, 8449783]
4947        Leo       Reise      2           [8448315, 8448317]
2218      Derek       Smith      2           [8451513, 8473953]
4002        Jim     Johnson      2           [8447099, 8448303]
2030       Dave        Reid      2           [8448307, 8450773]
7142       Ryan     Johnson      2           [8460527, 8481564]
7011        Ron    Anderson      2           [8444928, 8444929]
7673     Taylor        Hall      2           [8447463, 8475791]
1160      Brian       Smith      2           [8449061, 8449063]
5699       Mike       Green      2           [8468436, 8471242]
5706       Mike     Hoffman      2           [8447917, 8474884]
1628      Colin       White      2           [8465005, 8478400]
4140        Joe       Matte      2           [8447637, 8447638]
233   Alexandre      Picard      2      

In [ ]:
# Create a new column for full name
agg_df['FullName'] = agg_df['FirstName'] + ' ' + agg_df['LastName']

# Select the relevant columns
result_df = agg_df[['FullName', 'NumTeams', 'Teams']]

# Sort the DataFrame by the number of teams
sorted_df = result_df.sort_values(by='NumTeams', ascending=False)

# Display the sorted DataFrame
print(sorted_df.head(30).to_string(index=False))

         FullName  NumTeams                                                        Teams
   Mike Sillinger        12 [DET, VAN, CBJ, TBL, ANA, PHX, OTT, NYI, STL, FLA, PHI, NSH]
  J.J. Daigneault        10           [PIT, VAN, MTL, ANA, PHX, NYI, STL, PHI, NSH, MIN]
    Lee Stempniak        10           [PIT, NJD, CAR, NYR, WPG, PHX, CGY, TOR, STL, BOS]
  Derick Brassard        10           [ARI, PIT, CBJ, EDM, NYR, NYI, FLA, PHI, OTT, COL]
         Jim Dowd        10           [MIN, NJD, VAN, CHI, CGY, NYI, EDM, PHI, MTL, COL]
    Dominic Moore        10           [BUF, PIT, TBL, SJS, NYR, TOR, FLA, MTL, MIN, BOS]
Mathieu Schneider        10           [DET, ATL, LAK, VAN, ANA, NYR, PHX, TOR, NYI, MTL]
     Olli Jokinen        10           [LAK, NYR, WPG, PHX, CGY, TOR, NYI, STL, FLA, NSH]
     Michel Petit        10           [LAK, VAN, TBL, NYR, PHX, CGY, QUE, TOR, EDM, PHI]
       Tony Hrkac         9                [CHI, ATL, DAL, SJS, ANA, QUE, NYI, STL, EDM]
       Sean Burke    